# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおける「20の質問」ゲームをプレイするためのエージェントの実装を取り扱っています。特に、エージェントは質問を適切に作成し、推測を行う能力を持つことが求められています。

### 問題の概要
ノートブックでは、質問と回答のやりとりを通じて特定のターゲットワードを限定していくという、20の質問ゲームのルールに基づいて、参加者エージェントを構築しています。このゲームでは、プレイヤーは「はい」または「いいえ」で答えられる質問を通じて、対象を特定することが目的です。

### 使用している手法とライブラリ
- **分析ライブラリ**: NumPyとPandasがインポートされており、データ処理や線形代数の操作に使用されています。
- **Kaggle Environments**: Kaggle特有の環境を作成し、エージェントのインタラクションをシミュレートするために使用されています。
- **エージェントの設計**: 
    - `agent_first_letter`: 単語の最初の文字を基に推測を行うエージェント。ターンタイプによって条件を変え、質問や推測を生成するロジックが実装されています。
    - `simple_agent1`, `simple_agent2`, `simple_agent3`: 動物や固定の質問を返す簡易エージェント。これらは異なる戦略を用いて情報を収集し、推測を行います。

### 実行結果
ノートブックでは、これらのエージェントが相互に競い合う様子をシミュレーションし、結果をインタラクティブに表示します。`env.run`メソッドを使用してゲームを実行し、`env.render`メソッドで結果を可視化しています。

このノートブックでは、エージェントがどのように効率的に情報を処理し、推測を行うかという実装例を示しており、20の質問ゲームにおける戦略的思考や論理的推論を評価するための基盤を提供しています。

---


# 用語概説 
以下は、ジョブブック内のコードやその周辺に関連するコンテキストから、初心者がつまずきそうな専門用語の簡単な解説のリストです。

1. **エージェント (Agent)**:
   - エージェントは、特定のタスクを遂行するために設計されたプログラムやロジックのことを指します。このノートブックでは、質問をする役割の「質問者エージェント」と、答える役割の「回答者エージェント」の二つの役割があります。

2. **環境 (Environment)**:
   - エージェントが動作する空間や状況を指します。ここでは「llm_20_questions」環境が定義され、エージェント同士が「20の質問」ゲームを行います。

3. **観測 (Observation, obs)**:
   - エージェントが現在の状態を表すデータのことです。この観測には、現在のターンの種類や前の質問・回答などが含まれます。

4. **設定 (Configuration, cfg)**:
   - エージェントに与えられる設定情報のことです。環境のパラメータなどが含まれ、エージェントの動作を制御します。

5. **ターンタイプ (Turn Type)**:
   - ゲームの進行状態を示し、エージェントが行えるアクション（質問する、推測する、答える）を示すフィールドです。

6. **推測 (Guess)**:
   - エージェントがターゲットとなる単語を正確に当てようとする行為です。エージェントは「guess」ターンにおいて、この推測を行います。

7. **評価 (Eval)**:
   - プログラム内で式を計算し真偽を判定することを指します。特定の条件が真であるかどうかを判断するために使用されています。

8. **ランダム選択 (Random Choice)**:
   - 与えられた選択肢から無作為に1つを選ぶ処理です。アルゴリズムの多様性を持たせたり、単純なエージェントの行動に確率性を加えたりするために使われます。

9. **リスト (List)**:
   - 複数の元素を保持するデータ構造です。ここでは動物や人物、場所のリストが定義され、それらからランダムに選ぶために使用されています。

10. **CSVファイル (Comma-separated values)**:
    - データをコンマで区切って格納したファイル形式です。一般的にはテーブル形式のデータが保持され、`pandas`ライブラリを使用して読み込むことができます。

11. **Dockerイメージ (Docker Image)**:
    - アプリケーションとその依存関係を含むコンテナの構成ファイルです。Kaggleでは、特定の環境を再現するための標準基盤として使用されています。

12. **一時ファイル (Temporary File)**:
    - セッション中の一時的なデータを保持するファイルで、セッションが終了すると消去されます。競技環境での使用においてはデータの一時的な保存に用います。

これらの解説が、初心者がつまずきやすいポイントの理解に役立つことを期待しています。

---


このコードは、正しい答えを得るための新しい考え方を提供します。これはただのサンプルです。また、これが許可されているのかどうかも知りたいです。

In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています。
# これは、kaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、いくつかの便利なパッケージをロードする方法は以下の通りです。

import numpy as np # 線形代数
import pandas as pd # データ処理、CSVファイルI/O（例: pd.read_csv）

# 入力データファイルは、読み取り専用の"../input/"ディレクトリにあります。
# 例えば、これを実行すると（実行をクリックするか、Shift + Enterを押すことで）、入力ディレクトリ内のすべてのファイルがリスト表示されます。

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 現在のディレクトリ（/kaggle/working/）には最大20GB書き込むことができ、これは「すべてを保存して実行」することでバージョンを作成したときに出力として保存されます。
# 一時ファイルは/kaggle/temp/に書き込むこともできますが、現在のセッション外には保存されません。

In [ ]:
%%writefile submission/main.py

import os
import sys
import csv
import random

# 動物リスト、人物リスト、場所リストの定義
animal_list= ["dog","cat","hen","fish",'duck','cow','tiger','']
person_list= ['Elon Musk','Jos Biden','Jack Ma','Harry Potter']
place_list=['Beijing', "London", 'New York','Tokyo']

# 他のファイル（例：モデルの重み）を提出/libディレクトリに入れた場合、パスを設定する必要があります
KAGGLE_COMPETITION_PATH = "/kaggle_simulations/agent/" # コンペティションパス
if os.path.exists(KAGGLE_COMPETITION_PATH):  # コンペティション内で実行している場合
    subdirectory_path = os.path.join(KAGGLE_COMPETITION_PATH, "lib")
else: # ノートブックで実行している場合
    subdirectory_path = os.path.join("/kaggle/working/submission/", "lib")
sys.path.insert(0, subdirectory_path)

def agent_first_letter(obs, cfg):    
    if obs.turnType == "ask":        
        #print(obs.step)
        if len(obs['answers']) == 0:  
            response = "str(obs.keyword)[0].lower() in list('abcdefghijklm') #キーワードの最初の文字は次のいずれかですか: ['a','b','c','d','e','f','g','h','i','j','k','l','m']?"
        elif (len(obs.answers) == 1):
            if (obs.answers[-1]=="yes"):
                response = "str(obs.keyword)[0].lower() in list('abcdef') #キーワードの最初の文字は次のいずれかですか: ['a','b','c','d','e','f']?"
            else:
                response = "str(obs.keyword)[0].lower() in list('nopqrs') #キーワードの最初の文字は次のいずれかですか: ['n','o','p','q','r','s']?"
        elif (len(obs.answers) == 2):
            if (obs.answers[-2]=="yes")&(obs.answers[-1]=="yes"):
                response = "str(obs.keyword)[0].lower() in list('abc') #キーワードの最初の文字は次のいずれかですか: ['a','b','c'] ? "

            elif (obs.answers[-2]=="yes")&(obs.answers[-1]=="no"):
                response = "str(obs.keyword)[0].lower() in list('ghij') #キーワードの最初の文字は次のいずれかですか: ['g','h','i','j']?"

            elif (obs.answers[-2]=="no")&(obs.answers[-1]=="yes"):
                response = "str(obs.keyword)[0].lower() in list('nop') #キーワードの最初の文字は次のいずれかですか: ['n','o','p'] ?"

            elif (obs.answers[-2]=="no")&(obs.answers[-1]=="no"):
                response = "str(obs.keyword)[0].lower() in list('tuvw') #キーワードの最初の文字は次のいずれかですか: ['t','u','v','w'] ?"

        elif (len(obs.answers) == 3):
            if (obs.answers[-3]=="yes")&(obs.answers[-2]=="yes")&(obs.answers[-1]=="yes"):
                response = "str(obs.keyword)[0].lower() in list('ab') #キーワードの最初の文字は次のいずれかですか: ['a','b'] ?"

            elif (obs.answers[-3]=="yes")&(obs.answers[-2]=="yes")&(obs.answers[-1]=="no"):
                response = "str(obs.keyword)[0].lower() in list('de') #キーワードの最初の文字は次のいずれかですか: ['d','e'] ?"

            elif (obs.answers[-3]=="yes")&(obs.answers[-2]=="no")&(obs.answers[-1]=="yes"):
                response = "str(obs.keyword)[0].lower() in list('gh') #キーワードの最初の文字は次のいずれかですか: ['g','h'] ?"

            elif (obs.answers[-3]=="yes")&(obs.answers[-2]=="no")&(obs.answers[-1]=="no"):
                response = "str(obs.keyword)[0].lower() in list('kl') #キーワードの最初の文字は次のいずれかですか: ['k','l'] ?"

            elif (obs.answers[-3]=="no")&(obs.answers[-2]=="yes")&(obs.answers[-1]=="yes"):
                response = "str(obs.keyword)[0].lower() in list('no') #キーワードの最初の文字は次のいずれかですか: ['n','o'] ?"

            elif (obs.answers[-3]=="no")&(obs.answers[-2]=="yes")&(obs.answers[-1]=="no"):
                response = "str(obs.keyword)[0].lower() in list('qr') #キーワードの最初の文字は次のいずれかですか: ['q','r'] ?"

            elif (obs.answers[-3]=="no")&(obs.answers[-2]=="no")&(obs.answers[-1]=="yes"):
                response = "str(obs.keyword)[0].lower() in list('tu') #キーワードの最初の文字は次のいずれかですか: ['t','u'] ?"
            
            elif (obs.answers[-3]=="no")&(obs.answers[-2]=="no")&(obs.answers[-1]=="no"):
                response = "str(obs.keyword)[0].lower() in list('xz') #キーワードの最初の文字は次のいずれかですか: ['x','z'] ?"

        
        else:
            response = 'Is the keyword xxxxxx?' + str(obs["step"])
            
    # エージェントが推測者でターンタイプが"guess"のとき
    elif obs.turnType == "guess":
        
        if (obs.questions[-1]=="is it a place") & (obs.answers[-1]=="yes"):
            guess = random.choice(place_list) #場所リストからランダムに選ぶ
        else:
            guess = random.choice(animal_list) #動物リストからランダムに選ぶ
         
        response = guess     
  
    # エージェントが回答者でターンタイプが"answer"のとき
    elif obs.turnType == "answer":
        if "#" in (obs.questions)[-1]:
            ques = (obs.questions)[-1]
            try:
                ques = ques[0:ques.find('#')]            
                response = 'yes' if eval(ques) else 'no'  
            except:
                ques = ques[ques.find('#')+1:] 
                random.choice(['yes','no']) #ここでLLMを使用することもできます
        else:             
            response = random.choice(['yes','no']) #ここでLLMを使用することもできます        
    return response

In [ ]:
import random

def simple_agent1(obs, cfg):
    # エージェントが推測者でターンタイプが"ask"のとき
    if obs.turnType == "ask": 
        response = random.choice(list_qs) # 質問リストからランダムに選択
    elif obs.turnType == "guess": 
        # 前の答えが"yes"のとき、"duck"を推測し、そうでないときは"penguin"を推測
        if obs.answers[-1] == "yes":
            response = "duck"
        else:
            response = "penguin"
        
    elif obs.turnType == "answer":
        response = random.choice(['yes', 'no']) # ランダムに"yes"または"no"を返す
    return response

def simple_agent2(obs, cfg):
    # エージェントが推測者でターンタイプが"ask"のとき
    if obs.turnType == "ask": 
        response = "Is it a bird?" # 固定の質問を返す
    elif obs.turnType == "guess": 
        response = "bird" # "bird"と推測する
    elif obs.turnType == "answer":
        response = random.choice(['yes', 'no']) # ランダムに"yes"または"no"を返す
    return response

def simple_agent3(obs, cfg):
    # エージェントが推測者でターンタイプが"ask"のとき
    if obs.turnType == "ask": 
        response = random.choice(list_qs) # 質問リストからランダムに選択
    elif obs.turnType == "guess": 
        if len(obs.answers) == 0:
            response = "cat" # 最初の質問では"cat"を推測
        elif obs.answers[-1] == "yes":
            response = "duck" # 前の答えが"yes"のとき
        else:
            response = "penguin" # 前の答えが"no"のとき
        
    elif obs.turnType == "answer":
        # 最後の質問に"#"が含まれている場合
        if "#" in (obs.questions)[-1]:
            ques = (obs.questions)[-1]
            try:
                ques = ques[0:ques.find('#')]            
                response = 'yes' if eval(ques) else 'no'  
            except:
                response = random.choice(['yes', 'no']) # 予測できない場合はランダムに返す
        else:             
            response = random.choice(['yes', 'no']) # ここでLLMを使用することもできます
        
    return response

In [ ]:
from kaggle_environments import make

# llm_20_questions環境を作成
env = make(environment="llm_20_questions")  # 環境を設定し、20の質問ゲームを開始します。

In [ ]:
game_output = env.run(agents=[agent_first_letter, simple_agent3, simple_agent2, simple_agent1])
# ゲームを実行し、エージェントを指定して出力を取得します。
env.render(mode="ipython", width=600, height=900)
# ゲームの結果をIPythonノートブック内で表示します。ウィンドウの幅を600ピクセル、高さを900ピクセルに設定します。